## 2 Bit LSB Encoding
## This exercise implements a version of LSB steganography that replaces 2 LSBs from the audio file's bytes. It contains both coding and decoding algorithms
### The code has been adapted from a simple LSB implementation that replaces only the last bit

In [1]:
import wave

## Import sound file and convert it to a bytearray object

In [2]:
sound = wave.open ('sounds/Ex3_sound5.wav', mode = 'rb')
#convert file into a bytearray() object
byte_array = bytearray(list(sound.readframes(sound.getnframes())))

## Creating message and converting it to bits:
1. Create string to store message
2. Calculate the size difference, in bytes, of audio file and message
3. Multiplying the above size difference by 2 gives us the total extra bits our message needs
3. Divide this difference by 8 to get total extra bytes
4. Multiply the floor fo the above multiplication by the character '#', append this to message

In [12]:
# create string to store message
message ='Father Christmas does not exist'

# add '#' character to fill the empy spaces in the message.
# the length of the message in bits should be twice the length of the bytes_array
# since 2 bits in the message map to one byte in the original file
message= message + 2 * int((len(byte_array)-(len(message)*8*8))/8) *'#'
print(message[0:100])

Father Christmas does not exist#####################################################################


## Convert message to bits
1. Iterate over message
2. ord(i) gives us the ASCII representation of each character
3. bin(ord(i)) returns the binary for this ASCII
4. .lstrip('0b') removes the '0b' that appears at the start of each binary in python
5. .rjust replaces the removed '0b' with a 0 to give a proper 8 bit representation
6. ''.join creates a string of 1s and 0s
7. map converts them to integers
8. list() creates a list

In [14]:
# Convert message to a list of bits
bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8,'0') for i in message])))
print (bits[1:10])

[1, 0, 0, 0, 1, 1, 0, 0, 1]


## Replacing the last two LSBs
1. Iterate over bits with a step of 2
2. left shift bits[i] to insert it into the second last position
3. insert bits [i+1] at the end

In [6]:
for i in range (0, len(bits) - 1, 2):
    byte_array[i//2] = (byte_array[i//2] & 252) | (bits[i] << 1) | bits[i+1]
    
# Get the encoded bytes
encodedBytes = bytes(byte_array)

## Write bytes to a new file

In [7]:
# Write bytes to a new wave audio file
with wave.open('sounds/encoded1.wav', 'wb') as e:
    #writing audio to it
    e.setparams(sound.getparams())
    #writing the bytes to it
    e.writeframes(encodedBytes)
sound.close()

## Compare perceptual quality of original and encoded sounds

In [15]:
import thinkdsp
from thinkdsp import read_wave

In [16]:
original_sound = read_wave("sounds/Ex3_sound5.wav")
encoded_sound = read_wave ('sounds/encoded1.wav')

In [17]:
original_sound.make_audio()

In [18]:
encoded_sound.make_audio()

## We can see that there is a lot of noise in the encoded audio file

## Algorithm to decode message

In [21]:
#Decoding algorithm
def retrieve_message (audio_file_path):
    sound = wave.open(audio_file_path, mode='rb')
    #get the bytes from the audio file
    sound_bytes = sound.readframes(sound.getnframes())
    #convert bytes to list
    bytes_list = list(sound_bytes);
    #finally, create a bytearray object for modification
    byte_array = bytearray(bytes_list)
    
    # Extract the last 2 LSB of each byte
    lsb = []
    for byte in byte_array:
        # isolate two lsbs
        lsb_2 = byte & 3
        # shift second last bit to the right and extract
        bit1 = (lsb_2 >> 1) & 1 
        #extract last bit
        bit2 = lsb_2 & 1 
        lsb.append(bit1)
        lsb.append(bit2)
        
    #convert the list of lsb bytes to characters
    hidden_message = "".join(chr(int("".join(map(str,lsb[i:i+8])),2)) for i in range(0,len(lsb),8))
    
    # Remove empty characters
    hidden_message = hidden_message.split("###")[0]
    return hidden_message
   

In [22]:
decoded_message = retrieve_message("sounds/encoded1.wav")
print(decoded_message)

Father Christmas does not exist


### code adapted from: https://sumit-arora.medium.com/audio-steganography-the-art-of-hiding-secrets-within-earshot-part-2-of-2-c76b1be719b3